In [1]:
import ROOT
import numpy as np
# import toml
from annri_a1_analysis import config 
# from tqdm.notebook import tqdm, trange
from annri_a1_analysis import utils
import numba

%jsroot on

Welcome to JupyROOT 6.28/00


ModuleNotFoundError: No module named 'annri_a1_analysis'

## Load Config
The config holds all the information on how to construct the histograms and other general settings

In [2]:
# config = toml.load("stage0_config.toml")
config = config.load_config("stage0_config.toml")
config

## Read Calibration
Read the detector calibration from csv file

In [3]:
numch = config['general']['numch']

calib_slope = [1] * numch
calib_offset = [0] * numch

def read_calibration(filename, numch):
    
    with open(filename, "r") as file:
        
        file = open(filename, "r")
        file.readline()

        for line in file:
            values = line.strip().split(',')
            calib_slope[int(values[0])] = float(values[1])
            calib_offset[int(values[0])] = float(values[2])


        calib_slope = np.array(calib_slope)
        calib_offset = np.array(calib_offset)
        
read_calibration("calib/calib_apr.csv", 32)



In [ ]:
import csv
import numpy as np

def extract_columns_from_csv(filename, numch):
    # Initialize the arrays with default values
    calib_slope = np.ones(numch)
    calib_offset = np.zeros(numch)

    with open(filename, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header
        
        for row in reader:
            index = int(row[0])
            calib_slope[index] = float(row[1])
            calib_offset[index] = float(row[2])
            
    return calib_slope, calib_offset

# Usage
filename = "calib/calib_apr.csv"
numch = config['general']['numch']
calib_slope, calib_offset = extract_columns_from_csv(filename, numch)


In [ ]:
import csv
import numpy as np

def extract_calib_from_csv(filename, numch):
    # Initialize the arrays with default values
    calib_slope = np.ones(numch)
    calib_offset = np.zeros(numch)

    with open(filename, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header
        
        for row in reader:
            index = int(row[0])
            calib_slope[index] = float(row[1])
            calib_offset[index] = float(row[2])
            
    return calib_slope, calib_offset

# Usage
filename = "calib/calib_apr.csv"
numch = config['general']['numch']
calib_slope, calib_offset = extract_calib_from_csv(filename, numch)



## NUMBA Function
Create JIT compiled C++ function for creating hEgam column

In [4]:
@ROOT.Numba.Declare(["int","int"], "double")
def calc_Egam(detector,PulseHeight):
    return PulseHeight*calib_slope[detector]+calib_offset[detector]

## Create RDataFrame
Create data frame and define new columns then filter by channel. Create histogram models and then fill histograms.

In [ ]:
# ROOT.EnableImplicitMT()

# tree_name = config['general']['tree_name']
# numch = config['general']['numch']
# fp_length = config['general']['fp_length']
# pu_flag = config['general']['pu_flag']
# fo_flag = config['general']['fo_flag']

# # file_name = "data/rawroot_run_0064*"
# file_name = "data/rawroot_run_0014*"


# df = ROOT.RDataFrame(tree_name, file_name)


# s1 =  "PulseHeight != 0"
# s2 =  "PulseHeight == 0"
# s3 =  f"PulseHeight == 0 && Flags == {pu_flag}"
# s4 =  f"PulseHeight == 0 && Flags != {pu_flag}"
# s5 =  f"PulseHeight != 0 && Flags == {pu_flag}"
# s6 =  f"PulseHeight != 0 && Flags != {pu_flag}"
# s7 =  f"Flags == {pu_flag}"
# s8 =  f"Flags != {pu_flag}"

# df = df.Filter(f" detector == {1}")
# df1 = df.Filter(f" detector == {1} &&"+s1)
# df2 = df.Filter(f" detector == {1} &&"+s2)
# df3 = df.Filter(f" detector == {1} &&"+s3)
# df4 = df.Filter(f" detector == {1} &&"+s4)
# df5 = df.Filter(f" detector == {1} &&"+s5)
# df6 = df.Filter(f" detector == {1} &&"+s6)
# df7 = df.Filter(f" detector == {1} &&"+s7)
# df8 = df.Filter(f" detector == {1} &&"+s8)

# c = df.Count().GetValue()
# c1 = df1.Count().GetValue()
# c2 = df2.Count().GetValue()
# c3 = df3.Count().GetValue()
# c4 = df4.Count().GetValue()
# c5 = df5.Count().GetValue()
# c6 = df6.Count().GetValue()
# c7 = df7.Count().GetValue()
# c8 = df8.Count().GetValue()

# print("no cuts:" + f" {c} counts")
# print(s1 + f": {c1} counts {c1/c*100}%")
# print(s2 + f": {c2} counts {c2/c*100}%")
# print(s7 + f": {c7} counts {c7/c*100}%")
# print(s8 + f": {c8} counts {c8/c*100}%")
# print(s3 + f": {c3} counts {c3/c*100}%")
# print(s4 + f": {c4} counts {c4/c*100}%")
# print(s5 + f": {c5} counts {c5/c*100}%")
# print(s6 + f": {c6} counts {c6/c*100}%")

In [5]:
ROOT.EnableImplicitMT()

tree_name = config['general']['tree_name']
numch = config['general']['numch']
fp_length = config['general']['fp_length']
pu_flag = config['general']['pu_flag']
fo_flag = config['general']['fo_flag']

file_name = "data/rawroot_run_0014*"
# file_name = "data/rawroot_run_0064*"


df = ROOT.RDataFrame(tree_name, file_name)

df = (df.Define("tof_ns","tof*10.0")
      .Define("tof_mus","tof_ns/1000.0+0.0000001")
      .Define("En","pow((72.3*21.5/(tof_ns/1000.0)),2)+0.00001")
      .Define("Egam","Numba::calc_Egam(detector,PulseHeight)"))

df_ch = []

for ch in range(numch):
    
    df_ch.append(df.Filter(f"detector == {ch}"))

hist_dict = {}

for key, hist_conf in config['hist'].items():
    
    # name = hist_conf['name']
    xaxis = hist_conf['xaxis']
    yaxis = hist_conf['yaxis']
    
    col = hist_conf['col']
    gate = hist_conf['gate']
    
    down = hist_conf['bins']['down']
    up = hist_conf['bins']['up']
    N = hist_conf['bins']['N']
    var = hist_conf['bins']['var']

    
    df_gate_ch = []
    if gate != "":
        df_gate = df.Filter(gate)
        for ch in range(numch):
            df_gate_ch.append(df_gate.Filter(f"detector == {ch}"))
    else:
        df_gate = df
        df_gate_ch = df_ch
        

    if hist_conf['all']:

        name = hist_conf['name']
        title  = name + ';' + xaxis + ';' + yaxis
        xbins = utils.get_xbins(N, down, up, var, fp_length)

        hist_model_all = ROOT.RDF.TH1DModel(name, title, N, xbins)
        hist_all = df_gate.Histo1D(hist_model_all, col)

        hist_dict[key] = hist_all

        # if hist_conf['pu_rate']:
        #     hist_model_all_purate = ROOT.RDF.TH1DModel(name+"purate", title, N, xbins)
        #     hist_all_purate = df_gate.Histo1D(hist_model_all, col)
        #     hist_dict[key+"purate"] = hist_all_purate

            

    else:
        hist_dict[key] = {}
        for ch in range(numch):
            
            name = hist_conf['name'] + f'_d{ch}'
            title  = name + ';' + xaxis + ';' + yaxis
            xbins = utils.get_xbins(N, down, up, var, fp_length)

            if 'hEgam' in name:
                scale = calib_slope[ch]
            else:
                scale = 1

            hist_model_ch = ROOT.RDF.TH1DModel(name, title, N, xbins*scale)
            hist_ch = df_gate_ch[ch].Histo1D(hist_model_ch, col)

            hist_dict[key][name] = hist_ch
        
        
# hist_dict


## Write Histograms
Write histograms from dictionary to a root file maintaing heirerachy. 

In [6]:
# read_root_to_dict("test.root")

In [7]:
det_number = [-1,1,28,3,-1,7,6,2,-1,13,10,11,-1,9,-1,15,-1,18,17,19,-1,-1,-1,-1,-1,26,-1,-1,25,-1,-1,-1] # det number for index of position number runs 13~54
# det_number = [-1,1,28,3,-1,7,6,2,-1,13,10,11,-1,9,-1,15,-1,18,17,-1,19,-1,-1,-1,-1,26,-1,-1,25,-1,-1,-1] #det number for index of position number runs 55~90

# filename = "stage0_output.root"

det_pos_map = ROOT.TH2I("det_pos_map","det_pos_map;position # ;detector # ",32,0,32,32,0,32)
det_pos_graph = ROOT.TGraph()
det_pos_graph.SetTitle("det_pos_graph")
det_pos_graph.SetName("det_pos_graph")
det_pos_graph.GetXaxis().SetTitle("positon #")
det_pos_graph.GetYaxis().SetTitle("detector #")



for ch in range(numch):
    det_pos_map.SetBinContent(ch+1,det_number[ch]+1,1)
    det_pos_graph.AddPoint(ch,det_number[ch])

# hist_dict[det_pos_map.GetName()] = det_pos_map
# hist_dict[det_pos_graph.GetName()] = det_pos_graph


# with ROOT.TFile(filename,"UPDATE") as outfile:

#     if det_pos_graph.GetName() in outfile.GetListOfKeys():
#         # print(outfile.GetListOfKeys())
#         existing_histogram = outfile.Get(det_pos_map.GetName())
#         existing_histogram.Delete()

#     if det_pos_map.GetName() in outfile.GetListOfKeys():
#         existing_histogram = outfile.Get(det_pos_map.GetName())
#         existing_histogram.Delete()

#     # outfile.WriteObject(det_pos_graph, det_pos_graph.GetName())
    
#     det_pos_graph.Write()
#     det_pos_map.Write()

In [8]:
utils.write_dict_to_root(hist_dict, "stage0_output.root")

In [9]:
# read_root_to_dict("stage0_output.root")